In [28]:
import pandas as pd
from nltk.corpus import comparative_sentences  #keyword comparison 생성 모듈

data = pd.read_csv('reviews_preprocessing.csv', engine='python')
data.head()

,asin,date,product,review,score,user
0,B00000J48G,"11 18, 1999",Polaroid PDC-640 0.3MP Digital Camera Creative...,i have taken over 500 pics using all three foc...,4.0,A.J.Kirchoff (xgi@ametro.net)
1,B00000J48G,"11 18, 1999",Polaroid PDC-640 0.3MP Digital Camera Creative...,i wish that it would take the 16memory and th...,4.0,A.J.Kirchoff (xgi@ametro.net)
2,B00000J48G,"06 10, 2002",Polaroid PDC-640 0.3MP Digital Camera Creative...,only draw back is theres no optical lens.,4.0,"Andrea Rowland ""Andrea"""
3,B00000J48G,"04 26, 2000",Polaroid PDC-640 0.3MP Digital Camera Creative...,i like the way my 5x7 prints and smaller turn...,4.0,"""denisey65"""
4,B00000J48G,"04 26, 2000",Polaroid PDC-640 0.3MP Digital Camera Creative...,"flashing is important in any photography, and...",4.0,"""denisey65"""


In [127]:
#데이터 확인
len(data)

442911

In [128]:
#kw 사전 리스트 생성
### kw_list = comparative_sentences.keywords_readme()[5:]
### kw_str = '|'.join(kw_list)

#kw 사전 수작업으로 분류 후 다시 읽어오기
def call_kw():
    kw_list = pd.read_excel('kw_list.xls')
    #칼럼 중에 'to_use'만 가져오기
    kw_list = list(kw_list['to_use'])
#     kw_str = '|'.join(kw_list)
    
    return kw_list

for kw in call_kw():
    print(kw)

beat
inferior
outstrip
Choice
choose
prefer
recommend
outperform
superior
all
favor
defeat
twice
thrice
Number one
more
like
Versus
first
outdistance
before
double
outsell
nobody
Vs
last
after
thrice
improve
only
outmatch
ahead
fraction
outdo
unmatched
peerless
differ
outwit
Compare
top
exceed
lead
win
outstrip
none
unrivaled
dominate
second
nonpareil
advantage
unlike
least
outclass
outfox
outdistance
most
up against
less
half
behind
Compete


In [75]:
# kw_list=call_kw()
# #for문 기반 cs,ss 라벨링
# def make_sscs(data):
#     from nltk import word_tokenize, pos_tag
#     cs_list = []
#     ss_list = []
#     for i in range(len(data)):
#         text = data.loc[i, 'review']
#         product = data.loc[i, 'product']
        
#         # nltk로 토크나이즈, 포스태깅
#         tokens = word_tokenize(text)
#         pos_tokens = pos_tag(tokens)
        
#         # 비교구문이 들어있는 문장 찾기
#         count = 0
#         for tok, pos in pos_tokens:
#             if pos in ['JJR', 'JJS', 'RBS', "RBR"]:
#                 count +=1

#         if count != 0:
#             cs_list.append(i)
#         elif count == 0:
#             ss_list.append(i)
        
    
#     df_ss = data.loc[ss_list]     
    
#     df_cs = data.loc[cs_list]
#     # product_list가 없어서 #처리함. 추후 보강해야 함
# #     product_list = []
# #     products = '|'.join(product_list)
# #     df = df[df.review.str.contains(products)== True]
#     df_cs = df_cs[df_cs.review.str.contains(kw_str)== True]
    
#     df_cs['CS_SS'] = 'CS'
#     df_ss['CS_SS'] = 'SS'

#     df_t = df_cs.append(df_ss, ignore_index=True)
#     return df_t

In [129]:
#CS / SS 라벨링
kw_list=call_kw()
def tokenizedNtagging(text):
    #kw로 분류
    for kw in kw_list:
        if kw.lower() in text:
            return 'CS'
        
    #pos로 분류    
    from nltk import word_tokenize, pos_tag
    # nltk로 토크나이즈, 포스태깅
    tokens = word_tokenize(text)
    pos_tokens = pos_tag(tokens)

    count = 0
    for tok, pos in pos_tokens:
        if pos in ['JJR', 'JJS', 'RBS', "RBR"]:
            count+=1

    if count != 0:
        return 'CS'
    else:
        return 'SS'            

In [130]:
#모델 사전 호출 함수
def call_model():
    df=pd.read_csv('product_name_number.csv', engine='python')
    model_ls=list(df['model'])
    model_reg='( '+' | '.join(model_ls)+' )'
    return model_reg

In [131]:
#리뷰에서 언급된 모델명 탐색
import re
import numpy as np

model_reg = call_model()
def targetmodel_read(text):
    read_ls = re.findall(model_reg,text)
    if read_ls :
        return ','.join(read_ls)
    else:
        return np.nan

In [132]:
#csss 라벨링, 타겟 제품 검색 모델링
def make_sscs(data):
    data['CS_SS']=data['review']
    data['target']=data['review']
    data['CS_SS']=data['CS_SS'].apply(tokenizedNtagging)
    print('CS_SS 라벨링 끝')
    data_cs=data[data['CS_SS']=='CS']
    data_ss=data[data['CS_SS']=='SS']
    data_ss['target']=data_ss['target'].apply(lambda x:'NaN')
    data_cs['target']=data_cs['target'].apply(targetmodel_read)
    data_cs=data_cs.dropna()
    print('모델명 탐색 끝')
    new_data=pd.concat([data_cs,data_ss])
    return new_data

In [133]:
df_t = make_sscs(data)
df_t.head()

CS_SS 라벨링 끝


C:\Users\tjlee\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


모델명 탐색 끝


C:\Users\tjlee\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


,asin,date,product,review,score,user,CS_SS,target
75,B00000J48G,"06 8, 2002",Polaroid PDC-640 0.3MP Digital Camera Creative...,i don't recommend using either the medium or l...,3.0,Sam Bethune,CS,pdc 640
77,B00000J48G,"06 8, 2002",Polaroid PDC-640 0.3MP Digital Camera Creative...,the pixel mode you use will also make a differ...,3.0,Sam Bethune,CS,pdc 640
172,B00000JDEL,"06 24, 2000",Kodak DC240 1.2MP Digital Camera w/ 3x Optical...,the camera is so easy to use and has such a w...,4.0,Love2Dance,CS,dc240
205,B00000JFIF,"11 26, 1999",Olympus D-340R 1.2MP Digital Camera,"the olympus d-340r is, in my opinion, the best...",5.0,C. Petit,CS,d-340r
270,B00000JFIF,"01 2, 2003",Olympus D-340R 1.2MP Digital Camera,after a family mememoryer purchased this olymp...,1.0,"Joe Conley ""noevilpeople""",CS,d-340r


In [134]:
len(df_t[df_t.CS_SS=='CS'])

23852

In [135]:
len(df_t[df_t.CS_SS=='SS'])

179119

In [58]:
df_t.to_csv('labelingSSCS.csv',index=False, encoding='utf-8')

======================== 테스트 영역 ================================================

In [136]:
test_df2=df_t.copy()

In [144]:
test_df2[test_df2['target']==' xi ']

,asin,date,product,review,score,user,CS_SS,target
18814,B00005V619,"07 10, 2002",Minolta Dimage X 2MP Digital Camera with 3x Op...,perfect for family outings and friendly gather...,4.0,A. Hsieh,CS,xi
27528,B00006LV79,"11 12, 2002",Canon PowerShot S230 3.2 MP Digital ELPH Camer...,the picture quality is not quite as good (a bi...,5.0,"""clappingmonkey""",CS,xi
28068,B00006RYUX,"12 20, 2002",Minolta Dimage Xi 3.2MP Digital Camera w/ 3x O...,the dimage xi now lets you delete an image by ...,5.0,Ashegam,CS,xi
28080,B00006RYUX,"03 22, 2003",Minolta Dimage Xi 3.2MP Digital Camera w/ 3x O...,the xi has 50% more pixels and also more optic...,5.0,"Gadgester ""No Time, No Money""",CS,xi
28779,B000087HWM,"04 3, 2003",Pentax Optio S 3.2MP Digital Camera w/ 3x Opti...,compared before purchase with minolta dimage x...,5.0,"Randy Poe ""rpoe0728""",CS,xi
28780,B000087HWM,"04 3, 2003",Pentax Optio S 3.2MP Digital Camera w/ 3x Opti...,best feature is a focus focus for low-flash si...,5.0,"Randy Poe ""rpoe0728""",CS,xi
35478,B00008PVXA,"09 28, 2003",Remanufactured Minolta Dimage Xt 3.2 MP Digita...,but that competition has not stood still and t...,3.0,jemaerca,CS,xi
209066,B0029U0WWU,"06 2, 2012",Sony Alpha A230L 10.2 MP Digital SLR Camera wi...,i read a post that said the older lenses (befo...,4.0,D Allen,CS,xi


In [145]:
test_df2[test_df2['target']==' xi '].loc[28779,'review']

'compared before purchase with minolta dimage xi - optio s has more features, is smaller and flasher, and about the same price.'

In [138]:
test_df2['target'].value_counts()

NaN                                                 179119
 elph                                                  683
 interchangeable                                       491
 5d                                                    489
 7d                                                    326
 d7000                                                 321
 cmos                                                  310
 sigma                                                 283
 d90                                                   281
 s95                                                   241
 40d                                                   241
 mini                                                  233
 d700                                                  228
 d300                                                  216
 d800                                                  215
 60d                                                   209
 s100                                                  2

[' a ']